In [1]:
import pandas as pd
import numpy as np
import random
import os

from functools import reduce


# Questions: 
- how to add the dataset to gitignore?
- how can i create a separate module that holds my functions?


![Shark attacks, a project by Roberto Henríquez Perozo. Data Analytics Bootcamp at IronHack](shark-attacks.png)



<br><br>


<center>
    <h1> PART I: Data cleaning and exploration</h1>
</center>


## 🎣️ Step 0 - Basic knowledge
To begin the development of this project, it would be good to hold a minimum understanding of `Shark Attacks`.

As I did not know much about this topic at the day the project started, I have recurred to the shark-attack wiki: https://en.wikipedia.org/wiki/Shark_attack

With this information in mind, below is the process of data exploration, cleaning, and wrangling.


## 🎣️ Step 1 - Defining the dataset path, and importing it to begin basic dataset exploration

In [2]:
# To follow along and access the DataSet, download it from KAGGLE using this link
# https://www.kaggle.com/teajay/global-shark-attacks

# Once you have downloaded the DataSet, change the following `dataset` variable to match the 
# path where you have saved the 'attacks.csv' file.

dataset = 'attacks.csv' 
df = pd.read_csv(dataset, encoding='latin-1')

Now, we will check some basic information about the dataset, in order to formulate a more educated hypothesis which we could actually put to test with the data available.

Here, I notice that the shape of the `df` with no duplicates is very small when compared to the whole `df`.

## 🌊️  FUNCT
This comparison could be turned into its own function, as it will be executed quite often

In [3]:
# I'll delete de duplicated rows
print('before', df.shape)
df = df.drop_duplicates()
print('after', df.shape)

# And also take a look at the columns
df.columns

before (25723, 24)
after (6312, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [4]:
# I want to take a look at the time structures
df.Date

0        25-Jun-2018
1        18-Jun-2018
2        09-Jun-2018
3        08-Jun-2018
4        04-Jun-2018
            ...     
6307             NaN
6308             NaN
6309             NaN
8702             NaN
25722            NaN
Name: Date, Length: 6312, dtype: object

In [5]:
# I want to see what is the data on the last couple
# of columns which have unexplicit labels
df[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']]

,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...
6307,NaN,NaN,6309.0,NaN,NaN
6308,NaN,NaN,6310.0,NaN,NaN
6309,NaN,NaN,NaN,NaN,NaN
8702,NaN,NaN,NaN,NaN,NaN


In [6]:
# Too many null values on the last two columns... let's count them
print(df.shape)
df[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']].isnull().sum()

(6312, 24)


Case Number.1       10
Case Number.2       10
original order       3
Unnamed: 22       6311
Unnamed: 23       6310
dtype: int64

In [7]:
# If there is only 1 value in the 'Unnamed: 22' column, and 2 values in the
# 'Unnamed: 22' column, I'll not consider this data for my analysis.
df = df.drop(columns=['Unnamed: 22', 'Unnamed: 23'])

In [8]:
# Now we'll look at the columns again
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [9]:
# The following columns seemed a little bit rare, so i do a value count to find out what they are about
df['Case Number.1'].value_counts().sort_values()

2003.04.20.c    1
1861.02.12.R    1
1966.07.14      1
1979.03.00      1
2015.06.14.a    1
               ..
2006.09.02      2
2005.04.06      2
2009.12.18      2
1966.12.26      2
1923.00.00.a    2
Name: Case Number.1, Length: 6285, dtype: int64

In [10]:
df['Case Number.2'].value_counts().sort_values()

1983.07.05      1
1861.02.12.R    1
1966.07.14      1
1979.03.00      1
2015.06.14.a    1
               ..
2013.10.05      2
1923.00.00.a    2
2005.04.06      2
1990.05.10      2
1980.07.00      2
Name: Case Number.2, Length: 6286, dtype: int64

# 🌊️ Creating a Function

After some thought and googling, it seems like these are some sort of notation used to categorize and order the shark attacks.

They also use the a notation that includes dates. 
## Is it the same as the Date on the Date column?

To find out, I wantet to pick random samples of the dataframe, but python's built-in `random` module kept giving me trouble when I tried to use it alongside pandas.

In [11]:
# It's easier to make it into a function
""" 
OLDER VERSION OF SAMPLER FUNCTION:
def sampler(df, column, sample_size):
    # This function generates an iterator out of random rows from a pandas dataframe's specific column
    
    # Defining how many samples to fetch (this is the df index)
    for i in range(sample_size):
        
        # The 1000 value can be changed in future versions to match the size of the population
        e = random.choice(range(1000))
        yield f"index: {e}, sample: {df.iloc[e][column]}"
"""

' \nOLDER VERSION OF SAMPLER FUNCTION:\ndef sampler(df, column, sample_size):\n    # This function generates an iterator out of random rows from a pandas dataframe\'s specific column\n    \n    # Defining how many samples to fetch (this is the df index)\n    for i in range(sample_size):\n        \n        # The 1000 value can be changed in future versions to match the size of the population\n        e = random.choice(range(1000))\n        yield f"index: {e}, sample: {df.iloc[e][column]}"\n'

In [12]:
# It's easier to make it into a function
"""
VER 02
def sampler(df, column, sample_size):
# This function generates an iterator out of random rows from a pandas dataframe's specific column
    
    # Defining how many samples to fetch from the df
    for i in range(sample_size):
        
        # Now a random index is generated out of the total length of the column...
        i = random.choice(range(len(df[column])))
        # ... to return the data values in that index as an iterator:
        yield f"index: {i}, sample: {df.iloc[i][column]}"
        
        # For future versions, it would be good to look at how I can return the
        # data as a tupple with just the data, and not have it return a formatted string
        # or even better, a pandas dataframe with the results

# Now let's try it out
sampler(df, ['Date', 'Case Number.1', 'Case Number.2'], 10)
"""

'\nVER 02\ndef sampler(df, column, sample_size):\n# This function generates an iterator out of random rows from a pandas dataframe\'s specific column\n    \n    # Defining how many samples to fetch from the df\n    for i in range(sample_size):\n        \n        # Now a random index is generated out of the total length of the column...\n        i = random.choice(range(len(df[column])))\n        # ... to return the data values in that index as an iterator:\n        yield f"index: {i}, sample: {df.iloc[i][column]}"\n        \n        # For future versions, it would be good to look at how I can return the\n        # data as a tupple with just the data, and not have it return a formatted string\n        # or even better, a pandas dataframe with the results\n\n# Now let\'s try it out\nsampler(df, [\'Date\', \'Case Number.1\', \'Case Number.2\'], 10)\n'

In [13]:
# It's easier to make it into a function

#VER 03
def sampler(df, column, sample_size):
# This function generates an iterator out of random rows from a pandas dataframe's specific column
    
    # Defining how many samples to fetch from the df
    for i in range(sample_size):
        
        # Now a random index is generated out of the total length of the column...
        i = random.choice(range(len(df[column])))
        # ... to return the data values in that index as an iterator:
        yield df.iloc[i][column]
        
        # For future versions, it would be good to look at how I can return the
        # data as a tupple with just the data, and not have it return a formatted string
        # or even better, a pandas dataframe with the results


# Now let's try it out
sampler(df, ['Date', 'Case Number.1', 'Case Number.2'], 10)

<generator object sampler at 0x7fcbb5a7faf0>

In [14]:
###

# list(sampler(df, ['href', 'href formula'],10))

In [15]:
# Since sampler generates iterators, list() must be used to see its contents
list(sampler(df, ['Date', 'Case Number.1', 'Case Number.2'], 10))

[Date             08-Jan-2015
 Case Number.1     2015.01.08
 Case Number.2     2015.01.08
 Name: 459, dtype: object,
 Date              15-Aug-1959
 Case Number.1    1959.08.15.b
 Case Number.2    1959.08.15.b
 Name: 4069, dtype: object,
 Date              21-Sep-1931
 Case Number.1    1931.09.21.b
 Case Number.2    1931.09.21.b
 Name: 5003, dtype: object,
 Date             28-Apr-2001
 Case Number.1     2001.04.28
 Case Number.2     2001.04.28
 Name: 1952, dtype: object,
 Date             14-Oct-1969
 Case Number.1     1969.10.14
 Case Number.2     1969.10.14
 Name: 3419, dtype: object,
 Date             Before 2004
 Case Number.1        ND-0107
 Case Number.2        ND-0107
 Name: 6212, dtype: object,
 Date             10-Sep-1992
 Case Number.1     1992.09.10
 Case Number.2     1992.09.10
 Name: 2535, dtype: object,
 Date              22-Jan-2010
 Case Number.1    2010.01.22.b
 Case Number.2    2010.01.22.b
 Name: 1049, dtype: object,
 Date             16-Nov-1942
 Case Number.1    

## 🦈️
From this output, we can see that the `Case Number` Columnns are actually replicating the info that we already have on the `Date` column. Therefore, we will drop both `Case Number` columns

In [16]:
df = df.drop(columns=['Case Number.1', 'Case Number.2'])
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6309.0
6308,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6310.0
6309,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From this output, we notice that the last couple of rows are still holding many null values.


In [17]:
# but if we look closely, it's only the last 10 columns which have the nulls.
df.tail(15)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3.0
6301,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2.0
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0
6304,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0
6305,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6307.0
6306,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0


In [18]:
#since they are only 10 instances, we can drop them manually:
df = df.drop([6302, 6303,6304,6305,6306,6307,6308,6309,8702,25722])

#results
df.tail(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3.0
6301,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2.0


# 🦈️
Whith this cleaned dataframe, we can look deeper into the actual data.

Notice that we still have additional columns that are not giving us any **'meaty information'** !

In [19]:
# After checking, these are just old indexes
# df['original order'].value_counts()

# And these are only the titles of the pdfs
# df['pdf'].value_counts()

df = df.drop(columns = ['pdf', 'original order'])

# 🦈️
The `href` and `href formula` columsn look very similar, but since they can't be read on the DataFrame that pandas provides, we'll try to use our `sampler()` function again to compare them both

In [20]:
list(sampler(df, ['href', 'href formula'],2))
# This however, returns invalid links which are not accurately represented.
# example: 

[href            http://sharkattackfile.net/spreadsheets/pdf_di...
 href formula    http://sharkattackfile.net/spreadsheets/pdf_di...
 Name: 352, dtype: object,
 href            http://sharkattackfile.net/spreadsheets/pdf_di...
 href formula    http://sharkattackfile.net/spreadsheets/pdf_di...
 Name: 4965, dtype: object]

# 🦈️
To actually see the contents, I've resorted to two separate methods, `random.sample` and a `for` loop:

In [21]:
# With Random sample
display(random.sample(list(df['href']), 5))
display(random.sample(list(df['href formula']), 5))

['http://sharkattackfile.net/spreadsheets/pdf_directory/1934.01.07-Grant.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/2009.12.06-Large.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1959.01.17.a-Neil.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1830.07.26-JosephBlaney.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1987.08.15-Rogers_Collier.pdf']

['http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0064-Shatt-al-Arab.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1958.04.30-Bowers.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1946.11.20-EsronaJohnson.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1972.12.26-Pfosse.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/2006.06.15-Milea.pdf']

In [22]:
# With a FOR loop
for i in range(5):
    e = random.choice(range(1000))
    print(f"index: {e}, href:         {df.iloc[e]['href']}")
    print(f"index: {e}, href formula: {df.iloc[e]['href formula']}")

index: 274, href:         http://sharkattackfile.net/spreadsheets/pdf_directory/2016.05.22-Marcus.pdf
index: 274, href formula: http://sharkattackfile.net/spreadsheets/pdf_directory/2016.05.22-Marcus.pdf
index: 408, href:         http://sharkattackfile.net/spreadsheets/pdf_directory/2015.06.24.a-Kumpe.pdf
index: 408, href formula: http://sharkattackfile.net/spreadsheets/pdf_directory/2015.06.24.a-Kumpe.pdf
index: 30, href:         http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.14-Rowan.pff
index: 30, href formula: http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.14-Rowan.pff
index: 572, href:         http://sharkattackfile.net/spreadsheets/pdf_directory/2014.03.22.a-Hoffman.pdf
index: 572, href formula: http://sharkattackfile.net/spreadsheets/pdf_directory/2014.03.22.a-Hoffman.pdf
index: 708, href:         http://sharkattackfile.net/spreadsheets/pdf_directory/2013.01.21.R-Marshallsea.pdf
index: 708, href formula: http://sharkattackfile.net/spreadsheets/pdf_d

## 🦈️
The links on both columns seem to match, most of the times anyways.

In some cases, the `href` seems to have an duplication on its links which corrupted them and made them innaccessible.

However, the `href formula` actually saved the correct URL format.

In [23]:
print(df.iloc[332]['href']), print(df.iloc[332]['href formula'])
print()
print(df.iloc[324]['href']), print(df.iloc[324]['href formula'])
print()
print(df.iloc[588]['href']), print(df.iloc[588]['href formula'])
print()
print(df.iloc[569]['href']), print(df.iloc[569]['href formula'])

http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2015.11.15.a-Engelman.pdf
http://sharkattackfile.net/spreadsheets/pdf_directory/2015.11.15.a-Engelman.pdf

http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2015.12.21.a-Brazil.pdf
http://sharkattackfile.net/spreadsheets/pdf_directory/2015.12.21.a-Brazil.pdf

http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2014.00.00.b-OceanicWhitetip.pdf
http://sharkattackfile.net/spreadsheets/pdf_directory/2014.00.00.b-OceanicWhitetip.pdf

http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2014.04.03-Armstrong.pdf
http://sharkattackfile.net/spreadsheets/pdf_directory/2014.04.03-Armstrong.pdf


(None, None)

In [24]:
# for the sake of simplicity, we will drop the `href` column, and replace it with the `href formula`
df = df.drop(columns='href')

In [25]:
#The column with the name of the victims also does not bring much relevant information to our study
df = df.drop(columns='Name')
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ',
       'Investigator or Source', 'href formula'],
      dtype='object')

## 🏊️ IDEAS:
- The pdfs presented on the links seem quite structured
 
 - It could be possible to parse them later down the road and use a **REGEX** to find more data
 
 - Like, adding a column that lists the **'Moon Phase'** described on some of the pdfs

- I also have ran query a few times to notice that all pdfs have actually been uploaded to the same website and have the same naming structure

## 🦈️

Some column names can be simplified, and some have unnecesary white spaces.
 
 Let's fix that right away

In [26]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ',
       'Investigator or Source', 'href formula'],
      dtype='object')

In [27]:
df = df
df.columns = ['CaseNum', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex', 'Age', 'Injury', 'Fatal', 'Time',
       'Species', 'Source', 'href']
df.columns

Index(['CaseNum', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Source', 'href'],
      dtype='object')

## 🏊️ SPECIES IDEAS: 
- create a standardized column for species

## 🦈️
Many of the values from the Species column are `nulls`. We'll fill them with the same `Invalid` value that other cells already have

In [28]:
#BEFORE
df.Species.value_counts()

White shark                                           163
Shark involvement prior to death was not confirmed    105
Invalid                                               102
Shark involvement not confirmed                        88
Tiger shark                                            73
                                                     ... 
136-kg [300-lb] shark                                   1
9-foot shark                                            1
Tiger shark, 3.5 m                                      1
Spinner shark, 1.2 m to 1.5 m [4' to 5']                1
1.8 m [6'] shark, species identity questionable         1
Name: Species, Length: 1549, dtype: int64

In [29]:
df.Species = df.Species.fillna('Invalid')

#AFTER
df.Species.value_counts() 

Invalid                                               2940
White shark                                            163
Shark involvement prior to death was not confirmed     105
Shark involvement not confirmed                         88
Tiger shark                                             73
                                                      ... 
136-kg [300-lb] shark                                    1
9-foot shark                                             1
Tiger shark, 3.5 m                                       1
Spinner shark, 1.2 m to 1.5 m [4' to 5']                 1
1.8 m [6'] shark, species identity questionable          1
Name: Species, Length: 1549, dtype: int64

In [30]:
# With the .describe() method, we can see that there are 1549 unique values in this column
# It would be interesting to create a new column which narrows this down to less unique values.
df.Species.describe()

count        6302
unique       1549
top       Invalid
freq         2940
Name: Species, dtype: object

## 🦈️ 

[Can you guess the Pokémon?](https://i.ytimg.com/vi/mg1A94zBWBw/hqdefault.jpg)

There are more than 400 species of sharks, and while lurking around the `Species` column you can find all sort of weird animals. Did you know there's even one species of shark known as the *'Cookie Cutter shark'* ?

I certainly had no clue. 

Here I tried to sort a bit of the data, by creating a secondary column which filtered the different species, while also taking out possible confusions. The scrutiny for this categorization is quite laxed, as this is not a research paper, but more an exercise with data...

Below is the process of creating such a secondary table.

In [68]:
# Mapping the Species column to decrease the amount of unique values
df2 = df.copy()

df2['Species2'] = df2['Species']  # .map(lambda x: 'White shark' if 'White shark' in x else x)
df2.Species2.value_counts() 

Invalid                                               2940
White shark                                            163
Shark involvement prior to death was not confirmed     105
Shark involvement not confirmed                         88
Tiger shark                                             73
                                                      ... 
136-kg [300-lb] shark                                    1
9-foot shark                                             1
Tiger shark, 3.5 m                                       1
Spinner shark, 1.2 m to 1.5 m [4' to 5']                 1
1.8 m [6'] shark, species identity questionable          1
Name: Species2, Length: 1549, dtype: int64

# 🏊️ 

In [69]:
def shark_identifier(x):
    
    #THERE ARE SO MANY ERRORS, let's filter them
    
    if 'not confirmed' in x.lower():
        return "NOT CONFIRMED"
    if 'unidentified' in x.lower():
        return 'NOT CONFIRMED'
    if ' or ' in x.lower():
        return 'NOT CONFIRMED'
    
    if 'no shark involvement' in x.lower():
        return 'INVALID ENTRY'
    if 'invalid' in x.lower():
        return 'INVALID ENTRY'
    if 'questionable' in x.lower():
        return 'INVALID ENTRY'
    if 'doubtful' in x.lower():
        return 'INVALID ENTRY'
    
        
    if 'hoax' in x.lower():
        return 'HOAX'
    if 'drown' in x.lower():
        return 'DROWNED'
    if 'stingray' in x.lower():
        return 'STINGRAY'
    


    
    #  --- WHO'S THAT POKEMON?---
    
    if 'white shark' in x.lower():
        return "White shark"
    if 'tiger shark' in x.lower():
        return "Tiger shark"
    if 'bull shark' in x.lower():
        return "Bull shark"
    if 'nurse shark' in x.lower():
        return 'Nurse shark'
    if 'brown shark' in x.lower():
        return 'Brown shark'
    if 'mako shark' in x.lower():
        return 'Mako Shark'
    if 'blue shark' in x.lower():
        return 'Blue shark'
    if 'bronze whaler shark' in x.lower():
        return 'Bronze whaler shark'
    if 'blacktip shark' in x.lower():
        return 'Blacktip shark'
    if 'whitetip shark' in x.lower():
        return 'Whitetip shark'
    if 'sandbar shark' in x.lower():
        return 'Sandbar shark'
    if 'lemon shark' in x.lower():
        return 'Lemon shark'
    if 'hammerhead shark' in x.lower():
        return 'Hammerhead shark'
    if 'raggedtooth shark' in x.lower():
        return 'Raggedtooth shark'
    if 'thresher shark' in x.lower():
        return 'Thresher shark'
    if 'dusky shark' in x.lower():
        return 'Dusky shark'
    if 'wobbegong shark' in x.lower():
        return 'Wobbegong shark'
    if 'dusky shark' in x.lower():
        return 'Dusky shark'
    if 'spinner shark' in x.lower():
        return 'Spinner shark'
    if 'blue nose shark' in x.lower():
        return 'Blue nose shark'
    if 'leopard shark' in x.lower():
        return 'Leopard shark'
    if 'silvertip shark' in x.lower():
        return 'Silvertip shark'
    if 'gray shark' in x.lower():
        return 'Gray shark'
    if 'grey shark' in x.lower():
        return 'Gray shark'
    if 'reef shark' in x.lower():
        return 'Reef shark'
    if 'carpet shark' in x.lower():
        return 'Carpet shark'
    if 'whaler shark' in x.lower():
        return 'Whaler shark'
    
    # -- trying to filter sizes --
    
    if """10'""" in x.lower():
        return """10' shark"""
    if """11'""" in x.lower():
        return """11' shark"""
    if """12'""" in x.lower():
        return """12' shark"""
    if """13'""" in x.lower():
        return """13' shark"""
    if """14'""" in x.lower():
        return """14' shark"""
    if """15'""" in x.lower():
        return """15' shark"""
    if """16'""" in x.lower():
        return """16' shark"""
    if """17'""" in x.lower():
        return """17' shark"""
    if """18'""" in x.lower():
        return """18' shark"""
    if """19'""" in x.lower():
        return """19' shark"""
    if """20'""" in x.lower():
        return """20' shark"""
    if """21'""" in x.lower():
        return """21' shark"""
    
    if """1'""" in x.lower():
        return """1' shark"""
    if """2'""" in x.lower():
        return """2' shark"""
    if """3'""" in x.lower():
        return """3' shark"""
    if """4'""" in x.lower():
        return """4' shark"""
    if """5'""" in x.lower():
        return """5' shark"""
    if """6'""" in x.lower():
        return """6' shark"""
    if """7'""" in x.lower():
        return """7' shark"""    
    if """8'""" in x.lower():
        return """8' shark"""
    if """9'""" in x.lower():
        return """9' shark"""

    if 'small shark' in x.lower():
        return 'Small shark'
    
    else:
        return x
    
df2['Species2'] = df2['Species'].map(shark_identifier)

In [70]:
print(df2.Species2.value_counts().head(50))

INVALID ENTRY                                   3052
White shark                                      625
NOT CONFIRMED                                    305
Tiger shark                                      275
Bull shark                                       171
4' shark                                         131
5' shark                                         127
6' shark                                         124
Nurse shark                                       94
Shark involvement prior to death unconfirmed      68
3' shark                                          67
Reef shark                                        65
Bronze whaler shark                               60
Blacktip shark                                    56
Small shark                                       55
10' shark                                         54
Mako Shark                                        53
12' shark                                         47
Wobbegong shark                               

In [60]:
df2.groupby('Species2', 'Fatal').filter(lambda x : x > 2)

ValueError: No axis named Fatal for object type <class 'pandas.core.frame.DataFrame'>

## 🏊️ TYPE OF ATTACK
- On the Type column, dont count sea disasters, questionable and boatomg
- Stardarize
- Size of the shark according to Species column

In [ ]:
df.Type.isnull().sum()

In [ ]:
df.Type = df.Type.fillna('Invalid')

In [ ]:
df.Type.value_counts()

In [ ]:
df.Age.value_counts()

In [ ]:
df.columns

In [ ]:
df.Name.value_counts()

In [ ]:
df2 

## It looks still like some of these pdfs are duplicates, even after dropping duplicates :

In [ ]:
# how many times each pdf on the dataframe
df3 = df["href"]
df3.value_counts()

In [ ]:
# drop dupes and compare lengths
df_pdf_nodupes = df.drop_duplicates()

len(df) - len(df_pdf_nodupes), 'duped values'

In [ ]:
df.duplicated().sum()

In [ ]:
df.Country.value_counts()

In [ ]:
df['Fatal'].value_counts()

## Since the lengths are not the same, I will check if those duplicated entries are only in this column

### I'll look at the rest of the data now.

In [ ]:
remove_spaces = lambda x:  x.remove(' ') if ' ' in x else x
"""

df_label['Fatal'] = list(
                            map(remove_spaces(
                            df_label['Fatal']),
                            ))
"""
    
df_label['Fatal'].value_counts()


# I want to see the indexes which have a duplicated pdf row
"""

dupes = []
for a,b in list(df_label['pdf'].duplicated().items()):
    if b:
        dupes.append(a)
dupes """

In [ ]:
"""
df_label.loc[dupes]
"""


In [ ]:
# dfx = df_nodupes["pdf"].value_counts() if 

# Transform this to sort the shark species


In [ ]:
list(df_label['Species'].value_counts().items())

In [ ]:
# @@ Use this to fill null values: 
# df_clean["drive"] = df_clean.drive.fillna("NoTransmision")

# Injuries and types of attack
The GSAF categorizes scavenging bites on humans as "questionable incidents."

## PROVOKED
Provoked attacks occur when a human touches, hooks, nets, or otherwise aggravates the animal. Incidents that occur outside of a shark's natural habitat, such as aquariums and research holding-pens, are considered provoked, as are all incidents involving captured sharks. Sometimes humans inadvertently provoke an attack, such as when a surfer accidentally hits a shark with a surf board.

## UNPROVOKED
- Hit-and-run attack
- Sneak Attack
- Bump-and-bite attack 

For more information on how to differentiate PROVOKED vs UNPROVOKED attacks :
https://en.wikipedia.org/wiki/Shark_attack#Types_of_attacks

In [ ]:
# Since there is no column that states if the attack was provoked or not,
# I want to analyze the injury column to distinguish between the cases that were provoked
# and those that were unprovoked.

random.sample(list(df_label.Injury.value_counts().items()),20)

In [ ]:
# Categorizing  Provoked and  Unprovoked attacks
#df_clean.loc[df_clean["trany"].str.startswith("M"),"trany"] = "Manual"

provoked = ['PROVOKED', 'hook', 'shot']
#map(lambda words, x : words in x, provoked, df_nodupes.loc[df_nodupes['Injury'].str])
df_nodupes.loc[df_nodupes['Injury'].str]

In [ ]:
df_nodupes.loc[df_nodupes['Injury'].str]

# df_provoked = np.where(df_nodupes.Injury.isin(provoked), True, False) 

# Passing that categorization to a new PROVOKED COLUMN
df_nodupes['Provoked'] = df_provoked
df_nodupes['Provoked'] 

In [ ]:
display(df.columns) # To know which are the columns in the DF
display(df.count()) # To know how much data are we missin on each column
display(df.dtypes)

In [ ]:
df_label[['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Investigator or Source', 'pdf', 'href',
       'Case Number.1', 'Case Number.2']].head(50)